# Feature Engineering in BQ and BQML
 
In this Notebook we will do two exercises. First we will use standard ansi SQL to generate some basic feature engineering. For the second exercise will then use BigQuery ML to train a model and us the build in transformations. BigQuery ML enables users to create and execute machine learning models in BigQuery using SQL queries. The goal is to democratize machine learning by enabling SQL practitioners to build models using their existing tools and to increase development speed by eliminating the need for data movement.
 
We’ll use Airline On Time Performance data, a 70 million row data set from the U.S. Bureau of Transportation statistics, that is available to all users in BigQuery as the airline_ontime_data.flights data set

 
## Objectives
In this tutorial, you will:
 
+ Basic Feature Engineering using BigQuery
+ Feature Engineering BigQuery Machine Learning (BQML). 
+ Use BigQuery ML Transform to prepare a dataset that can be used for modeling. 
+ Create and train your first BigQuery model. 
+ Evaluate Model the model trained. 

## Feature generation using BigQuery. 
 
We have already seen in our previous Data Analysis exercises that we can extract a feature from a `CAST`:
 
+ EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day
+ FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week
 
You can also use SQL to do things like replacing missing values (`NULL` in this case). 
 
+ IFNULL(airline, 'N/A') AS NA
 
You can see how it works by running the following example. 

In [4]:
%%bigquery --verbose

SELECT 
    CAST(date AS DATE) AS flight_date, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week, 
    airline,
    departure_delay,
    arrival_delay,
    IFNULL(airline, 'N/A') AS NA,
    IFNULL(arrival_delay, 0) AS label
FROM 
    `bigquery-samples.airline_ontime_data.flights`
LIMIT 5

Executing query with job ID: ec8e65f8-d437-4d76-8d70-2128e0427cfb
Query executing: 0.59s
Query complete after 1.38s


,flight_date,flight_day,flight_day_of_week,airline,departure_delay,arrival_delay,NA,label
0,2008-04-13,13,Sun,MQ,14.0,1.0,MQ,1.0
1,2008-04-11,11,Fri,MQ,3.0,4.0,MQ,4.0
2,2008-04-10,10,Thu,MQ,94.0,124.0,MQ,124.0
3,2011-11-17,17,Thu,XE,-2.0,-7.0,XE,-7.0
4,2008-04-09,9,Wed,MQ,28.0,30.0,MQ,30.0


## Exercise one: Create a new target feature. 

Now if we want to build a classification model that predicts if a flight will have a delay, meaning departure and/or arrival delay. Because we want to do a classification we need to generate a new feature: delayed (Yes / No) based on the features `departure_delay` and `arrival_delay`. Let's try to write a query that generates our new target feature that we can use in our model. 

To do:
+ Writing a query generates a new feature called delayed that can have the value `Yes` or `No`.
+ Set it to `Yes` if there is a delay ( > 0) for `departure_delay` or `arrival_delay`.
+ Feel free to select any other features you like as long as you generate the new feature.
+ Tip: Use the [BigQuery GUI](https://console.cloud.google.com/bigquery) to write, run and debug your query. 

In [5]:
%%bigquery --verbose

SELECT 
    CAST(date AS DATE) AS flight_date, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week, 
    airline,
    departure_delay,
    arrival_delay,
    IF(arrival_delay > 0 OR departure_delay > 0, "Delay", "No_delay") AS Delayed
FROM 
    `bigquery-samples.airline_ontime_data.flights`
LIMIT 10

Executing query with job ID: 38d1bd7c-78a8-4b65-a57a-2ca3db515608
Query executing: 0.33s
Query complete after 2.73s


,flight_date,flight_day,flight_day_of_week,airline,departure_delay,arrival_delay,Delayed
0,2007-10-30,30,Tue,OO,-5.0,-7.0,No_delay
1,2009-02-27,27,Fri,OO,13.0,23.0,Delay
2,2009-02-23,23,Mon,OO,-3.0,-3.0,No_delay
3,2009-02-24,24,Tue,OO,7.0,24.0,Delay
4,2007-10-08,8,Mon,OO,48.0,112.0,Delay
5,2009-02-20,20,Fri,OO,14.0,32.0,Delay
6,2009-02-25,25,Wed,OO,-3.0,-15.0,No_delay
7,2007-11-08,8,Thu,OO,-6.0,-25.0,No_delay
8,2007-11-07,7,Wed,OO,-4.0,-19.0,No_delay
9,2007-11-06,6,Tue,OO,3.0,-10.0,Delay


## Please stop here :)

In [ ]:
# Only run if needed

from google.cloud import bigquery

client = bigquery.Client(location="US")

## Exercise two: Build our first BigQuery ML model to predict delay 

Let's build a model using just the raw data. It's not going to be very good, but sometimes it is good to actually experience this. The model will take a bit of time to train. When it comes to ML, this is very fast.

To do:
+ Write your own model use the code below as a starting point. 
+ Tip: Use the [console](https://console.cloud.google.com/) to write, run and debug your query. 
+ Have look at the [BQML documentation](https://cloud.google.com/bigquery-ml/docs/bigqueryml-web-ui-start). 
+ Use as your target: departure_delay or arrival_delay.
+ Make sure you take a random sample.

In [ ]:
--bigquery --verbose

CREATE OR REPLACE MODEL bqml_tutorial.model_basic
OPTIONS(input_label_cols=['Delayed'], model_type='logistic_reg') AS
SELECT 
    airline,
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    EXTRACT(YEAR FROM CAST(date AS DATE)) AS flight_year,
    departure_delay,
    arrival_delay,
    departure_airport,
    arrival_airport,
    IF(arrival_delay > 0 OR departure_delay > 0, "Delay", "No_delay") AS Delayed
FROM `bigquery-samples.airline_ontime_data.flights`
WHERE RAND() < 0.001

## Exercise three: Build a more advanced model 

Now it's time to build a more complex model and add a BigQuery ML native transformation.
Using the `TRANSFORM` clause, you can specify all preprocessing during model creation. The preprocessing is automatically applied during the prediction and evaluation phases of machine learning. 

To do:
+ Copy and past the first BigQuery Model you created and add:
    + Add extra ML OPTION to the model. Just try a few and see what happens.
    + Implement BigQuery ML native TRANSFORM (choose any feature). Have a look at the [documentation](https://cloud.google.com/bigquery-ml/docs/bigqueryml-transform)
+ Use as your target the same as in exercise 2.
+ Use a random sample. 
+ Train on data from: 2006 AND 2011
+ Tip: Use the [console](https://console.cloud.google.com/) to write, run and debug your query.  

In [6]:
%%bigquery --verbose

CREATE OR REPLACE MODEL bqml_tutorial.model_transform
TRANSFORM(
          flight_date,
          flight_day,
          airline, 
          flight_month,
          flight_year,
          departure_airport,
          arrival_airport,
          arrival_delay,
          ML.MIN_MAX_SCALER(departure_delay) OVER() as buckets
          )
OPTIONS(input_label_cols=['arrival_delay'],  DATA_SPLIT_METHOD='AUTO_SPLIT', OPTIMIZE_STRATEGY='BATCH_GRADIENT_DESCENT', model_type='linear_reg') AS
SELECT 
    flight_date,
    airline,
    flight_day, 
    flight_month,
    flight_year,
    departure_delay,
    arrival_delay,
    departure_airport,
    arrival_airport
FROM 
    (SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    EXTRACT(YEAR FROM CAST(date AS DATE)) AS flight_year,
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week,
    airline, 
    departure_airport,
    arrival_airport,
    departure_delay,
    arrival_delay
    FROM `bigquery-samples.airline_ontime_data.flights`)
WHERE flight_year BETWEEN 2006 AND 2011
AND RAND() < 0.001

Executing query with job ID: bdb0598c-368c-4bb1-8992-e8bd50e0d54f
Query executing: 33.23s
Query complete after 33.88s


""


## Exercise four: Model evaluation. 

Now it's time to evaluate the model you just trained. 
 
To do:
+ Write the full code for doing model evaluation.
+ Have look at the [BQML documentation](https://cloud.google.com/bigquery-ml/docs/bigqueryml-web-ui-start). 
+ Make sure to use the same features as that you trained on. 
+ Use data from a different time period then where you trained on (Tip: use the where clause). 
+ Use a random sample. 
+ Tip: Use the [console](https://console.cloud.google.com/) to write, run and debug your query. 

In [2]:
%%bigquery --verbose

SELECT * FROM ML.EVALUATE(MODEL bqml_tutorial.model_basic,
(
SELECT 
    flight_date,
    airline,
    flight_day, 
    flight_month,
    flight_year,
    departure_delay,
    arrival_delay,
    departure_airport,
    arrival_airport, 
    Delayed
FROM 
    (SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    EXTRACT(YEAR FROM CAST(date AS DATE)) AS flight_year,
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week,
    airline, 
    departure_airport,
    arrival_airport,
    departure_delay,
    arrival_delay,
    IF(arrival_delay > 0 OR departure_delay > 0, "Delay", "No_delay") AS Delayed
    FROM `bigquery-samples.airline_ontime_data.flights`)
WHERE flight_year=2012
AND RAND() < 0.001
))

Executing query with job ID: 9b0ccd15-0024-461c-b82f-6f58a50d0b41
Query executing: 23.66s
Query complete after 25.36s


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.800906,0.906945,0.829174,0.850634,0.40518,0.903147


Or if you want to select a specific metric:

In [4]:
%%bigquery --verbose

SELECT accuracy FROM ML.EVALUATE(MODEL bqml_tutorial.model_basic,
(
SELECT 
    flight_date,
    airline,
    flight_day, 
    flight_month,
    flight_year,
    departure_delay,
    arrival_delay,
    departure_airport,
    arrival_airport, 
    Delayed
FROM 
    (SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    EXTRACT(YEAR FROM CAST(date AS DATE)) AS flight_year,
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week,
    airline, 
    departure_airport,
    arrival_airport,
    departure_delay,
    arrival_delay,
    IF(arrival_delay > 0 OR departure_delay > 0, "Delay", "No_delay") AS Delayed
    FROM `bigquery-samples.airline_ontime_data.flights`)
WHERE flight_year=2012
AND RAND() < 0.001
))

Executing query with job ID: 59610b75-3302-4752-818a-0d4d9d0ed2cc
Query executing: 1.72s
Query complete after 2.20s


,accuracy
0,0.837455


Or if you want to generate a confusion matrix.

In [6]:
%%bigquery --verbose

SELECT * FROM ML.CONFUSION_MATRIX(MODEL bqml_tutorial.model_basic,
(
SELECT 
    flight_date,
    airline,
    flight_day, 
    flight_month,
    flight_year,
    departure_delay,
    arrival_delay,
    departure_airport,
    arrival_airport, 
    Delayed
FROM 
    (SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    EXTRACT(YEAR FROM CAST(date AS DATE)) AS flight_year,
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week,
    airline, 
    departure_airport,
    arrival_airport,
    departure_delay,
    arrival_delay,
    IF(arrival_delay > 0 OR departure_delay > 0, "Delay", "No_delay") AS Delayed
    FROM `bigquery-samples.airline_ontime_data.flights`)
WHERE flight_year=2012
AND RAND() < 0.001
))

Executing query with job ID: 102944e4-a049-4466-a944-7340c8557f6b
Query executing: 5.24s
Query complete after 6.76s


,expected_label,Delay,No_delay
0,Delay,1907,640
1,No_delay,303,2644


Disclaimer: The goal of this exercise is learning more about how BigQuery ML can be used. The goal is not to train the best model but feel free to improve the model performance :) 

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
